## Colin's active development sandbox

In [10]:
from enum import Enum
from pathlib import Path
import logging
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import os
import re
import matplotlib.pyplot as plt
import cv2
from PIL import Image

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] - [%(levelname)s]: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

class DataPaths(Enum):
    ROOT_DATA_DIR = str(Path.cwd().parent / 'data' / 'SoccerNet' / 'jersey-2023' / 'extracted')
    TEST_DATA_DIR = str(Path(ROOT_DATA_DIR) / 'test' / 'images')
    TRAIN_DATA_DIR = str(Path(ROOT_DATA_DIR) / 'train' / 'images')
    VALIDATION_DATA_DIR = str(Path(ROOT_DATA_DIR) / 'challenge' / 'images')
    TEMP_EXPERIMENT_DIR = str(Path.cwd() / 'experiments' / 'temp')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"ROOT_DATA_DIR: {DataPaths.ROOT_DATA_DIR.value}")
logging.info(f"TRAIN_DATA_DIR: {DataPaths.TRAIN_DATA_DIR.value}")
logging.info(f"TEST_DATA_DIR: {DataPaths.TEST_DATA_DIR.value}")
logging.info(f"VAL_DATA_DIR: {DataPaths.VALIDATION_DATA_DIR.value}")
logging.info(f"Using device: {device}")


[2025-02-26 10:31:32] - [INFO]: ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
[2025-02-26 10:31:32] - [INFO]: TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
[2025-02-26 10:31:32] - [INFO]: TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
[2025-02-26 10:31:32] - [INFO]: VAL_DATA_DIR: c:\Users\colin\OneDrive\Desktop\UBC\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
[2025-02-26 10:31:32] - [INFO]: Using device: cuda


### Tracklet iterator

In [11]:
# Ignore the .DS_Store files

def get_tracks(input_folder):
    tracks = [t for t in os.listdir(input_folder) if not t.startswith('.')]
    logging.info(tracks[0:10])

    # Extract numerical part and convert to integer for comparison
    def extract_number(track):
        match = re.search(r'(\d+)', track)  # Extracts the first sequence of digits
        if match:
            return int(match.group(1))
        return -1  # Provide a default value if no number is found

    # Find min and max tracklets based on the extracted number
    if tracks:
        min_track = min(tracks, key=extract_number)
        max_track = max(tracks, key=extract_number)

        logging.info(f"Min tracklet: {min_track}")
        logging.info(f"Max tracklet: {max_track}")
    else:
        logging.warning("No tracklets found.")
        
    return tracks

In [12]:
def generate_features(input_folder, output_folder, model_version='res50_market', load_only=False):
    """
    If load_only is True, simply load images, apply transforms, and return the tensors.
    Otherwise, run them through the model backbone and return the extracted features.
    """
    if not load_only:
        # Load model if we're going to extract features
        CONFIG_FILE, MODEL_FILE = get_specs_from_version(model_version)
        cfg.merge_from_file(CONFIG_FILE)
        opts = ["MODEL.PRETRAIN_PATH", MODEL_FILE, "MODEL.PRETRAINED", True,
                "TEST.ONLY_TEST", True, "MODEL.RESUME_TRAINING", False]
        cfg.merge_from_list(opts)
        use_cuda = True if torch.cuda.is_available() and cfg.GPU_IDS else False
        model = CTLModel.load_from_checkpoint(cfg.MODEL.PRETRAIN_PATH, cfg=cfg)
        if use_cuda:
            model.to('cuda')
            logging.info("using GPU")
        model.eval()
    else:
        use_cuda = False  # No model inference when load_only is True

    # Define validation transforms using torchvision
    val_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Get list of valid track directories (skip hidden files)
    tracks = get_tracks(DataPaths.TRAIN_DATA_DIR.value)[0]
    
    # Dictionary to store processed data (per track)
    processed_data = {}
    for track in tqdm(tracks, desc="Processing tracks"):
        track_path = os.path.normpath(os.path.join(input_folder, track))
        if not os.path.isdir(track_path):
            continue
        images = [img for img in os.listdir(track_path) if not img.startswith('.')]
        track_features = []
        for img_path in images:
            img_full_path = os.path.normpath(os.path.join(track_path, img_path))
            try:
                # Load image using cv2 then convert to PIL
                img = cv2.imread(img_full_path)
                if img is None:
                    continue
                input_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                # Apply the validation transforms to get a tensor
                transformed = val_transforms(input_img)  # returns a tensor
                if load_only:
                    # Simply store the tensor (add a batch dimension for later concatenation)
                    track_features.append(transformed.unsqueeze(0))
                else:
                    # If not load_only, run through model to extract features
                    input_tensor = torch.stack([transformed])
                    with torch.no_grad():
                        _, global_feat = model.backbone(input_tensor.cuda() if use_cuda else input_tensor)
                        global_feat = model.bn(global_feat)
                    # Flatten and convert to numpy
                    track_features.append(global_feat.cpu().numpy().reshape(-1,))
            except Exception as e:
                logging.info(f"Error processing {img_full_path}: {e}")
                continue
        if track_features:
            if load_only:
                # Concatenate tensors along the batch dimension
                processed_data[track] = torch.cat(track_features, dim=0)
            else:
                processed_data[track] = np.array(track_features)
    return processed_data

In [ ]:
input_folder = DataPaths.TRAIN_DATA_DIR.value

# Call generate_features with load_only=True to simply load the images as tensors
data_dict = generate_features(input_folder, DataPaths.TEMP_EXPERIMENT_DIR.value, load_only=True)

# Inspect the loaded data
for track, tensor in data_dict.items():
    logging.info(f"Track: {track}, Loaded Tensor Shape: {tensor.shape}")

# Visualize a sample image from one track
import matplotlib.pyplot as plt
sample_track = list(data_dict.keys())[0]
sample_tensor = data_dict[sample_track][0]  # take the first image in the track
# Convert tensor to numpy and show it
plt.imshow(sample_tensor.permute(1, 2, 0).numpy())
plt.title(f"Sample from track: {sample_track}")
plt.show()

[2025-02-26 10:31:32] - [INFO]: ['0', '1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005']
[2025-02-26 10:31:32] - [INFO]: Min tracklet: 0
[2025-02-26 10:31:32] - [INFO]: Max tracklet: 1426


Processing tracks:   0%|          | 0/1 [00:00<?, ?it/s]